Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import nltk
nltk.download('omw-1.4')
import nltk
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
import nltk
nltk.download('words')
from nltk.corpus import words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.metrics import plot_confusion_matrix, classification_report
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
import lightgbm as lgb

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


ImportError: ignored

In [ ]:
!pip install lightgbm

Loading the data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("ind.csv")

Raw Exploration of Data

In [ ]:
df

,Sr. No.,Script,Label
0,1,"Kuch samay aur aage aajao, Sati bhi hota tha b...",1
1,2,The fact that the women in the past didn't use...,1
2,3,Madam plz read holy Quran and holy Bhagavad Gi...,0
3,4,So evolution doesn't mean anything? It was a c...,0
4,5,Wese to sab pehele bandar the...to wese hi ghu...,1
...,...,...,...
295,296,Bhai Nepal mae it’s damn expensive rs 1000 :(,0
296,297,"bohot khushnaseebi hamaari, sway your big fat ...",1
297,298,"machines dekh ke maja aa gaya,, errr you wr al...",0
298,299,"only in power have the rights,, human or inhuman",0


Exploratory Data Analytics

In [ ]:
df['Label'].unique()

array([1, 0])

This shows that our label is either 0 or 1.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Sr. No.  300 non-null    int64 
 1   Script   300 non-null    object
 2   Label    300 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 7.2+ KB


This shows there are no missing values in our dataset.
There is one independent section-Script and one dependent section-Label.

In [ ]:
print(df.duplicated().sum())

0


No duplicate script is present.

In [ ]:
df.head()

,Sr. No.,Script,Label
0,1,"Kuch samay aur aage aajao, Sati bhi hota tha b...",1
1,2,The fact that the women in the past didn't use...,1
2,3,Madam plz read holy Quran and holy Bhagavad Gi...,0
3,4,So evolution doesn't mean anything? It was a c...,0
4,5,Wese to sab pehele bandar the...to wese hi ghu...,1


Removing stopwords

In [ ]:
from gensim.parsing.preprocessing import remove_stopwords

In [ ]:
def stopword_removal(row):
  text = row['Script']
  text = remove_stopwords(text)
  return text

In [ ]:
df['Script']=df.apply(stopword_removal, axis=1)

In [ ]:
df.head()

,Sr. No.,Script,Label
0,1,"Kuch samay aur aage aajao, Sati bhi hota tha b...",1
1,2,The fact women past didn't use bare bodies car...,1
2,3,Madam plz read holy Quran holy Bhagavad Gita c...,0
3,4,So evolution doesn't mean anything? It common ...,0
4,5,Wese sab pehele bandar the...to wese hi ghume ...,1


Removing the punctuations and converting the upper case to lower case

In [ ]:
df['Script'] = df['Script'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')

<ipython-input-16-c087b53307ac>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Script'] = df['Script'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')


In [ ]:
df.head()

,Sr. No.,Script,Label
0,1,kuch samay aur aage aajao sati bhi hota tha bh...,1
1,2,the fact women past didn t use bare bodies car...,1
2,3,madam plz read holy quran holy bhagavad gita c...,0
3,4,so evolution doesn t mean anything it common t...,0
4,5,wese sab pehele bandar the to wese hi ghume ya...,1


Data Preprocessing


1. Tokenization

In [ ]:
def length(text):
  return len(word_tokenize(text))
df['word_count']=df['Script'].apply(length)

In [ ]:
df.head()

,Sr. No.,Script,Label,word_count
0,1,kuch samay aur aage aajao sati bhi hota tha bh...,1,16
1,2,the fact women past didn t use bare bodies car...,1,41
2,3,madam plz read holy quran holy bhagavad gita c...,0,18
3,4,so evolution doesn t mean anything it common t...,0,25
4,5,wese sab pehele bandar the to wese hi ghume ya...,1,27


2. Lemmatization


In [ ]:
def lemmatize(Words):
    new_words = []
    lem = WordNetLemmatizer()
    for w in Words:
      new_words.append(lem.lemmatize(w))
    return new_words

for w in df['Script']:
  if w in words.words():
    lemmatize(w)




In [ ]:
df.head()

,Sr. No.,Script,Label,word_count
0,1,kuch samay aur aage aajao sati bhi hota tha bh...,1,16
1,2,the fact women past didn t use bare bodies car...,1,41
2,3,madam plz read holy quran holy bhagavad gita c...,0,18
3,4,so evolution doesn t mean anything it common t...,0,25
4,5,wese sab pehele bandar the to wese hi ghume ya...,1,27


Feature Extraction Techniques

BoW- Bag of Words

In [ ]:
import nltk
def tokenize(text):
  return (word_tokenize(text))
df['tokenize']=df['Script'].apply(tokenize)

Create a BoW Model

In [ ]:
word2count = {}
for data in df['tokenize']:
    for i in data:
      if i not in word2count.keys():
            word2count[i] = 1
      else:
            word2count[i] += 1


print(word2count)


{'kuch': 4, 'samay': 1, 'aur': 17, 'aage': 1, 'aajao': 1, 'sati': 1, 'bhi': 14, 'hota': 5, 'tha': 3, 'bharat': 2, 'mein': 10, 'woh': 4, 'kyun': 1, 'nahi': 14, 'kar': 13, 'leti': 1, 'the': 9, 'fact': 4, 'women': 7, 'past': 1, 'didn': 2, 't': 28, 'use': 2, 'bare': 1, 'bodies': 1, 'career': 4, 'it': 25, 'you': 14, 'hand': 2, 'jiska': 1, 'tharkiyo': 1, 'ke': 24, 'bina': 1, 'chal': 3, 'hi': 12, 'nhi': 12, 'sakta': 2, 'agar': 4, 'adhnanga': 1, 'ghumna': 1, 'tumhe': 2, 'talent': 3, 'lagta': 10, 'hai': 104, 'toh': 16, 'khule': 1, 'mei': 1, 'sauch': 1, 'karna': 3, 'usse': 1, 'bada': 2, 'hua': 1, 'madam': 1, 'plz': 3, 'read': 2, 'holy': 2, 'quran': 1, 'bhagavad': 1, 'gita': 1, 'commenting': 1, 'issue': 1, 'we': 6, 'indians': 1, 'tolerate': 1, 'misguided': 1, 'tweets': 2, 'misinformation': 1, 'request': 1, 'refrain': 1, 'so': 7, 'evolution': 1, 'doesn': 4, 'mean': 3, 'anything': 1, 'common': 5, 'thing': 2, 'society': 5, 'moved': 1, 'attire': 1, 'try': 2, 'different': 4, 'challenges': 1, 'morality

Tf-Idf

In [ ]:
df

,Sr. No.,Script,Label,word_count,tokenize
0,1,kuch samay aur aage aajao sati bhi hota tha bh...,1,16,"[kuch, samay, aur, aage, aajao, sati, bhi, hot..."
1,2,the fact women past didn t use bare bodies car...,1,41,"[the, fact, women, past, didn, t, use, bare, b..."
2,3,madam plz read holy quran holy bhagavad gita c...,0,18,"[madam, plz, read, holy, quran, holy, bhagavad..."
3,4,so evolution doesn t mean anything it common t...,0,25,"[so, evolution, doesn, t, mean, anything, it, ..."
4,5,wese sab pehele bandar the to wese hi ghume ya...,1,27,"[wese, sab, pehele, bandar, the, to, wese, hi,..."
...,...,...,...,...,...
295,296,bhai nepal mae it s damn expensive rs 1000,0,9,"[bhai, nepal, mae, it, s, damn, expensive, rs,..."
296,297,bohot khushnaseebi hamaari sway big fat lib as...,1,9,"[bohot, khushnaseebi, hamaari, sway, big, fat,..."
297,298,machines dekh ke maja aa gaya errr wr invisible,0,9,"[machines, dekh, ke, maja, aa, gaya, errr, wr,..."
298,299,power rights human inhuman,0,4,"[power, rights, human, inhuman]"


In [ ]:
df_tf_Idf= df['Script']

In [ ]:
# create object
tfidf = TfidfVectorizer()

# get tf-df values
result = tfidf.fit_transform(df_tf_Idf)
print(result)

  (0, 1120)	0.28686969909163657
  (0, 998)	0.19758359611263118
  (0, 1295)	0.19404860074029145
  (0, 1072)	0.28686969909163657
  (0, 2010)	0.2431620735453045
  (0, 1231)	0.21512429191444365
  (0, 247)	0.267528766677045
  (0, 1824)	0.2538061567436269
  (0, 848)	0.23446522432903527
  (0, 253)	0.19404860074029145
  (0, 1635)	0.28686969909163657
  (0, 23)	0.28686969909163657
  (0, 21)	0.28686969909163657
  (0, 150)	0.19075759878270315
  (0, 1622)	0.28686969909163657
  (0, 1063)	0.2431620735453045
  (1, 856)	0.16601614226719047
  (1, 178)	0.15482323134809312
  (1, 1933)	0.16601614226719047
  (1, 1007)	0.14688173466790255
  (1, 1636)	0.16601614226719047
  (1, 1230)	0.16601614226719047
  (1, 1031)	0.16601614226719047
  (1, 1864)	0.2172258389386533
  (1, 778)	0.06418420088551371
  :	:
  (297, 604)	0.3697018536728183
  (297, 1186)	0.3697018536728183
  (297, 1173)	0.3697018536728183
  (297, 19)	0.3133738753771577
  (297, 714)	0.29268960980270775
  (297, 487)	0.29268960980270775
  (297, 1015)	0.2

In [ ]:
# get idf values
print('\nidf values:')
for ele1, ele2 in zip(tfidf.get_feature_names(), tfidf.idf_):
    print(ele1, ':', ele2)


idf values:
1000 : 6.01396308418893
11 : 5.608497976080766
15th : 6.01396308418893
1992 : 6.01396308418893
1998 : 6.01396308418893
2018 : 6.01396308418893
2024 : 6.01396308418893
21 : 6.01396308418893
24 : 6.01396308418893
25 : 6.01396308418893
25th : 6.01396308418893
26 : 6.01396308418893
30 : 6.01396308418893
300 : 6.01396308418893
46000cr : 6.01396308418893
50 : 6.01396308418893
500 : 6.01396308418893
5g : 6.01396308418893
89k : 6.01396308418893
aa : 5.097672352314776
aadha : 6.01396308418893
aage : 6.01396308418893
aaj : 5.608497976080766
aajao : 6.01396308418893
aajkal : 6.01396308418893
aake : 6.01396308418893
aandhbhakt : 6.01396308418893
aao : 6.01396308418893
aap : 4.915350795520821
aapne : 5.608497976080766
aate : 5.3208159036289855
aati : 5.608497976080766
aaur : 6.01396308418893
aaye : 5.608497976080766
aayega : 6.01396308418893
ab : 4.761200115693562
abba : 6.01396308418893
abcd : 6.01396308418893
abe : 4.509885687412656
abey : 6.01396308418893
abhi : 5.608497976080766
ab

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


6.01396308418893
badega : 6.01396308418893
badh : 6.01396308418893
badhiya : 6.01396308418893
badi : 6.01396308418893
badly : 6.01396308418893
bahut : 6.01396308418893
baithnewala : 6.01396308418893
baki : 6.01396308418893
bana : 6.01396308418893
banadiya : 6.01396308418893
banake : 6.01396308418893
banar : 6.01396308418893
bandar : 6.01396308418893
bandh : 6.01396308418893
bandhe : 6.01396308418893
banjau : 6.01396308418893
bank : 5.608497976080766
banned : 6.01396308418893
bano : 6.01396308418893
bap : 6.01396308418893
baqi : 6.01396308418893
bare : 6.01396308418893
barkha : 6.01396308418893
bars : 6.01396308418893
bas : 4.761200115693562
basically : 6.01396308418893
basics : 6.01396308418893
bataiye : 6.01396308418893
batao : 6.01396308418893
bataraha : 6.01396308418893
bate : 6.01396308418893
bath : 6.01396308418893
bbc : 6.01396308418893
bbcdocumentry : 6.01396308418893
bc : 5.608497976080766
bcho : 6.01396308418893
bcz : 6.01396308418893
be : 5.3208159036289855
beautiful : 5.6084

In [ ]:
# get indexing
print('\nWord indexes:')
print(tfidf.vocabulary_)

# display tf-idf values
print('\ntf-idf value:')
print(result)

# in matrix form
print('\ntf-idf values in matrix form:')
print(result.toarray())


Word indexes:
{'kuch': 1063, 'samay': 1622, 'aur': 150, 'aage': 21, 'aajao': 23, 'sati': 1635, 'bhi': 253, 'hota': 848, 'tha': 1824, 'bharat': 247, 'mein': 1231, 'woh': 2010, 'kyun': 1072, 'nahi': 1295, 'kar': 998, 'leti': 1120, 'the': 1830, 'fact': 631, 'women': 2011, 'past': 1400, 'didn': 516, 'use': 1929, 'bare': 201, 'bodies': 276, 'career': 322, 'it': 927, 'you': 2036, 'hand': 784, 'jiska': 961, 'tharkiyo': 1828, 'ke': 1015, 'bina': 263, 'chal': 342, 'hi': 813, 'nhi': 1318, 'sakta': 1609, 'agar': 76, 'adhnanga': 64, 'ghumna': 727, 'tumhe': 1895, 'talent': 1796, 'lagta': 1086, 'hai': 778, 'toh': 1864, 'khule': 1031, 'mei': 1230, 'sauch': 1636, 'karna': 1007, 'usse': 1933, 'bada': 178, 'hua': 856, 'madam': 1174, 'plz': 1445, 'read': 1537, 'holy': 834, 'quran': 1505, 'bhagavad': 241, 'gita': 734, 'commenting': 403, 'issue': 925, 'we': 1980, 'indians': 884, 'tolerate': 1866, 'misguided': 1254, 'tweets': 1899, 'misinformation': 1255, 'request': 1561, 'refrain': 1545, 'so': 1710, 'evol

In [ ]:
x = tfidf.vocabulary_
len(x)

2135

Splitting the data into train and test

In [ ]:
df

In [ ]:
X = df['Script']
y = df['Label']

# split the dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=0)


In [ ]:
print("1", X_train)
print("2", X_test)
print("3", y_train)
print("4", y_test)

In [ ]:
tfidf = TfidfVectorizer(max_features = 270)  # Using the TF - IDF Vectorizer to extract top 5000 most important features

In [ ]:
# Feature Extraction
X_train_tfidf = tfidf.fit_transform(X_train)  # Creating the vocabulary only from the training set to avoid data leakage from
X_test_tfidf = tfidf.transform(X_test)        # the test set.

In [ ]:
X_train_tfidf  # Sparse Matrix is created to save memory since many values are close to 0

In [ ]:
X_test_tfidf  # Sparse Matrix

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
tfidf_array_train = X_train_tfidf.toarray()   # Converting the sparse matrix to a numpy array (dense matrix)
tfidf_array_test = X_test_tfidf.toarray()     # Converting the sparse matrix to a numpy array (dense matrix)
scaled_X_train = scaler.fit_transform(tfidf_array_train)  # Fitting on only training data to avoid data leakage from test data
scaled_X_test = scaler.transform(tfidf_array_test) # and then tranforming both training and testing data

Performing Dimensionality Reduction using Principal Component Analysis

In [ ]:

from sklearn.decomposition import PCA
NUM_COMPONENTS = 270   # Total number of features
pca = PCA(NUM_COMPONENTS)
reduced = pca.fit(scaled_X_train)


In [ ]:
variance_explained = np.cumsum(pca.explained_variance_ratio_)  # Calculating the cumulative explained variance by the components

In [ ]:
# Plotting
fig, ax = plt.subplots(figsize=(8, 6))
plt.plot(range(NUM_COMPONENTS),variance_explained, color='r')
ax.grid(True)
plt.xlabel("Number of components")
plt.ylabel("Cumulative explained variance")

In [ ]:
final_pca = PCA(0.9)
reduced_90 = final_pca.fit_transform(scaled_X_train) # Number of Components explaining 90% variance in the training data

In [ ]:
reduced_90_test = final_pca.transform(scaled_X_test)

In [ ]:
reduced_90.shape

141 components explain 90% variance of data

In [ ]:
final_pca = PCA(0.8)
reduced_80 = final_pca.fit_transform(scaled_X_train) # Number of Components explaining 80% variance in the training data

In [ ]:
reduced_80.shape

110 components explain 80 % variance of data.

#Model Training

1. Random Forest


In [ ]:
# RANDOM FORESTS
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(random_state = 42)
n_estimators = [64, 100, 128]
bootstrap = [True, False] # Bootstrapping is true by default
param_grid = {'n_estimators': n_estimators, 'bootstrap': bootstrap}
grid_rf_model = HalvingGridSearchCV(rf_model, param_grid = param_grid, n_jobs = -1, min_resources = 'exhaust', factor = 3)
grid_rf_model.fit(X_train_tfidf, y_train)
preds_grid_rf_model = grid_rf_model.predict(X_test_tfidf)
print(classification_report(y_test, preds_grid_rf_model))
plot_confusion_matrix(grid_rf_model, X_test_tfidf, y_test)

2. Logistic Regression


In [ ]:
# LOGISTIC REGRESSION with the the 90% variance data
from sklearn.linear_model import LogisticRegression
log_model_pca = LogisticRegression()
log_model_pca.fit(reduced_90, y_train)
preds_log_model_pca = log_model_pca.predict(reduced_90_test)
print(classification_report(y_test, preds_log_model_pca))
plot_confusion_matrix(log_model_pca, reduced_90_test, y_test)

In [ ]:
# LOGISTIC REGRESSION with the complete data
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
log_model = LogisticRegression(solver = 'saga')
param_grid = {'C': np.logspace(0, 10, 5)}
grid_log_model = HalvingGridSearchCV(log_model, param_grid = param_grid, n_jobs = -1, min_resources = 'exhaust', factor = 3)
grid_log_model.fit(X_train_tfidf, y_train)
preds_grid_log_model = grid_log_model.predict(X_test_tfidf)
print(classification_report(y_test, preds_grid_log_model))
plot_confusion_matrix(grid_log_model, X_test_tfidf, y_test)

3. Stochastic Gradient Descent

In [ ]:
sgd_model = SGDClassifier()
sgd_model.fit(reduced_90,y_train)
preds_sgd_model = sgd_model.predict(reduced_90_test)
print(classification_report(y_test, preds_sgd_model))
plot_confusion_matrix(sgd_model, reduced_90_test, y_test)

4. Light GBM


In [ ]:
cll = lgb.LGBMClassifier()
cll.fit(reduced_90, y_train)
# predict the results
y_pred=cll.predict(reduced_90_test)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))
plot_confusion_matrix(cll, reduced_90_test, y_test)

Pipeline


In [ ]:
# Creating a pipeline
from sklearn.pipeline import Pipeline
pipe1 = Pipeline([('tfidf', TfidfVectorizer(max_features = 270)), ('rf_model', RandomForestClassifier(n_estimators = 128, random_state = 42))])
pipe1.fit(X, y)

In [ ]:
x = str(input())
pipe1.predict([x])

In [ ]:
# Creating a pipeline
from sklearn.pipeline import Pipeline
pipe2 = Pipeline([('tfidf', TfidfVectorizer(max_features = 270)), ('log_model_pca', LogisticRegression())])
pipe2.fit(X, y)

In [ ]:
v = str(input())
pipe2.predict([v])

In [ ]:
# Creating a pipeline
from sklearn.pipeline import Pipeline
pipe3 = Pipeline([('tfidf', TfidfVectorizer(max_features = 270)), ('sgd_model', SGDClassifier())])
pipe3.fit(X, y)

In [ ]:
z = str(input())
pipe3.predict([z])

In [ ]:
# Creating a pipeline
from sklearn.pipeline import Pipeline
pipe4 = Pipeline([('tfidf', TfidfVectorizer(max_features = 270)), ('cll', lgb.LGBMClassifier())])
pipe4.fit(X, y)

In [ ]:
w = str(input())
pipe4.predict([w])